# Model Rozpoznawania Encji Nazwanych (NER) - Trenowanie i Inferencja
#### Wprowadzenie
Ten notebook Jupyter dokumentuje proces trenowania modelu do rozpoznawania encji nazwanych (NER) przy użyciu wytrenowanego wcześniej modelu BERT do klasyfikacji tokenów. W kodzie użyta została biblioteka transformers do pracy z modelami BERT, pandas do obsługi danych, oraz scikit-learn do implementacji prostego klasyfikatora bazowego.

#### Zależności
Upewnij się, że masz zainstalowane poniższe biblioteki przed uruchomieniem kodu:

- transformers
- torch
- pandas
- scikit-learn
#### Dane
Dane treningowe są wczytywane z pliku JSONL zawierającego przykłady tekstu i odpowiadające im adnotacje encji.

In [ ]:
{"id":33,"text":"Bank Handlowy w Warszawie S.A. z siedzibą w Warszawie, ul. Senatorska 16, 00-923 Warszawa, zarejestrowany w rejestrze przedsiębiorców Krajowego Rejestru Sądowego przez Sąd Rejonowy dla m.st. Warszawy w Warszawie, XII Wydział Gospodarczy Krajowego Rejestru Sądowego, pod nr. KRS 000 000 1538; NIP 526-030-02-91; wysokość kapitału zakładowego wynosi 522 638 400 złotych, kapitał został w pełni opłacony. Citi Handlowy, Citigold, CitiOne, CitiKonto są zastrzeżonymi znakami towarowymi należącymi do podmiotów z grupy Citigroup Inc. 01\/2023Tabela Oprocentowania Kont Obowiązuje od 30 stycznia 2023 r. Oprocentowanie na Koncie Oszczędnościowym PLN USD GBP 3,00% 0,01% 0,01% Oprocentowanie na Koncie SuperOszczędnościowym PLN USD GBP do 20 000 6,00% 0,05% 0,05% powyżej 20 000 3,00% 0,00% 0,00% Oprocentowanie na Koncie Osobistym PLNUSD, GBP, CHF, EUR, AUD, CAD, ZAR, SEK, NOK, DKK, CZK, HUF 0,00% 0,00%",
 "label":[[615,638,"PRODUCT_TYPE"],[651,656,"PERCENTAGE"],[657,662,"PERCENTAGE"],[663,668,"PERCENTAGE"],[687,693,"PRODUCT_TYPE"],[694,715,"OFFER_NAME"],[728,737,"MAX_DEPOSIT"],[738,743,"PERCENTAGE"],[756,770,"MAX_DEPOSIT"],[771,776,"PERCENTAGE"],[777,782,"PERCENTAGE"],[783,788,"PERCENTAGE"],[807,813,"OFFER_TYPE"]],"Comments":[]}

#### Inicjalizacja Modelu
Wczytany zostaje wytrenowany wcześniej model BERT dla języka polskiego wraz z odpowiadającym mu tokenizatorem. Model jest skonfigurowany do klasyfikacji tokenów z określoną liczbą etykiet (encji).

In [ ]:
model = BertForTokenClassification.from_pretrained("dkleczek/bert-base-polish-uncased-v1", num_labels=6)
tokenizer = BertTokenizerFast.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

#### Przygotowanie Danych
Funkcja prepare_data jest zdefiniowana do przekształcania surowego tekstu i adnotacji encji na tokeny, etykiety i maski uwagi odpowiednie do trenowania modelu BERT.

In [ ]:
def prepare_data(text, annotations, tokenizer, max_length):
    # ... (szczegóły implementacji znajdują się w ostatnim akapicie pt. kod źródłowy)
    return tokens, labels, attention_mask


#### Trenowanie Modelu
Następnie kod przystępuje do trenowania modelu BERT przy użyciu przygotowanych danych. Używany jest prosty pętla trenująca z optymalizatorem AdamW.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
for epoch in range(5):
    for _, row in train_data_df.iterrows():
        # ... (szczegóły pętli trenującej znajdują się w ostatnim akapicie pt. kod źródłowy)

#### Pipeline Rozpoznawania Encji Nazwanych (NER)
W końcu tworzony jest pipeline NER do dokonywania predykcji na nowych przykładach tekstu.

In [ ]:
nlp_ner = pipeline('ner', model=model, tokenizer=tokenizer)

# Przykład wykorzystania:

tekst_przykładowy = "Przykładowy tekst do NER."
encje = nlp_ner(tekst_przykładowy)

Wyuczony model NLP pozawala systemowi wyłuskiwać tylko i wyłączenie słowa kluczowe dotyczące lokat i rachunków oszczędnościowych z plików .pdf zawierających oferty banków dostępnych na ich stronach internetowych. Dzięki współpracy z modelem klasyfikacyjnym jest w stanie z dużą dokładnością wyłuskać treść ofert depozytowych nadającą się do analizy w środowiskach programistycznych.

#### Kod źródłowy:

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification, pipeline
import torch
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import os

# Ścieżki do danych
FULL_TRAINING_SET_FILE = "D:\Projekty\Modules\scrapper\data\dataset.jsonl"

# Wczytanie danych treningowych
full_train_df : pd.DataFrame = pd.read_json(FULL_TRAINING_SET_FILE)
train_data_df = full_train_df[:20]
print(train_data_df.head())

# Funkcja przygotowująca dane treningowe
def prepare_data(text, annotations, tokenizer, max_length):
    encoded = tokenizer.encode_plus(text, return_offsets_mapping=True, max_length=max_length, padding='max_length', truncation=True)
    tokens = encoded['input_ids']
    attention_mask = encoded['attention_mask']
    offsets = encoded['offset_mapping']

    labels = [-100] * max_length

    # Mapowanie adnotacji na poziomie znaków do poziomu tokenów
    for start_char, end_char, label in annotations:
        start_token = None
        end_token = None
        for token_index, (start, end) in enumerate(offsets):
            if (start_token is None) and (start <= start_char) and (end >= start_char):
                start_token = token_index
            if (end_token is None) and (start <= end_char) and (end >= end_char):
                end_token = token_index
                break
        if start_token is not None and end_token is not None:
            labels[start_token:end_token + 1] = [1] * (end_token - start_token + 1)  # Ustawienie etykiet

    return tokens, labels, attention_mask

# Wczytanie pre-trenowanego modelu BERT dla klasyfikacji tokenów
model = BertForTokenClassification.from_pretrained("dkleczek/bert-base-polish-uncased-v1", num_labels=6)  # num_labels to liczba klas encji
tokenizer = BertTokenizerFast.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

# Proces trenowania modelu
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
for epoch in range(5):
    for _, row in train_data_df.iterrows():
        text = row["text"]
        annotations = row["label"]
        tokens, labels, attention_mask = prepare_data(text, annotations, tokenizer, max_length=512)
        inputs = {
            "input_ids": torch.tensor([tokens]),
            "attention_mask": torch.tensor([attention_mask]),
            "labels": torch.tensor([labels])
        }
        model.train()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Utworzenie pipeline do rozpoznawania encji
nlp_ner = pipeline('ner', model=model, tokenizer=tokenizer)